In [1]:
import numpy
import matplotlib.pyplot as plt
import matplotlib
import scipy.special
import scipy.misc
from scipy.ndimage import rotate
from scipy.misc import face
import PIL
from PIL import Image
from IPython.display import clear_output
import socket
import threading
import os, sys
import random

%matplotlib inline

start_position = numpy.array([
                    ['♜','♞','♝','♛','♚','♝','♞','♜'],
                    ['♟','♟','♟','♟','♟','♟','♟','♟'],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    ['♙','♙','♙','♙','♙','♙','♙','♙'],
                    ['♖','♘','♗','♕','♔','♗','♘','♖']])

reverse_start_position = numpy.rot90(start_position, 2)
path = "D:\Catalog\Проекты\Web\OpenServer\OSPanel\domains\Chess\icons/"

def resize():
    for item in os.listdir(path):
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((75,75), Image.ANTIALIAS)
            imResize.save(f + '.png', 'PNG', quality=100)
            

def toString(pos):
    s = ""
    for arr in pos:
        for sym in arr:
            s += sym
    return s
def toPos(s):
    pos = numpy.array([[' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' ']])
    for i in range(8):
        for j in range(8):
            pos[i][j] = s[8*i+j]
    return pos
def isWhite(fig):
    if fig in "♙♖♘♗♕♔":
        return True
    return False

def isBlack(fig):
    if fig in "♟♜♞♝♛♚":
        return True
    return False

In [2]:
class chess:
    def __init__(self, vis = True, start_pos = start_position, prop = [False, False, False, False, False, False, (7, 4), (0, 4), True, [(-1, -1), (-1, -1)]]):
        self.pos = start_pos.copy()
        self.last = prop[9]
        self.white = prop[8]
        self.white_figs = []
        self.black_figs = []
        self.updateFigs()
        self.vis = vis
        self.white_king = prop[0]
        self.white_rook_right = prop[1]
        self.white_rook_left = prop[2]
        self.black_king = prop[3]
        self.black_rook_right = prop[4]
        self.black_rook_left = prop[5]
        self.white_king_coords = prop[6]
        self.black_king_coords = prop[7]
        self.history = []
        self.prop = prop
        if vis:
            print(self.pos)
    def getPos(self):
        return self.pos
    def getReversePos(self):
        return numpy.rot90(self.pos, 2)
    def board(self, cur):
        return self.pos[cur[0]][cur[1]]
    def toArray(self, pos, white_figs, black_figs):
        res = numpy.array([0.01]*82*7)
        i = 0
        figs1 = "♙♖♘♗♕♔"
        figs2 = "♟♜♞♝♛♚"
        for k in range(0, len(figs1)):
            i = 8*7*k
            for (x, y) in white_figs:
                if pos[x][y] == figs1[k]:
                    b = "{0:b}".format(8*x+y+1)
                    for j in range(0, len(b)):
                        if b[j] == '1':
                            res[i] = 0.99
                        i+=1
                        j+=1
        i = 41*7
        for k in range(0, len(figs2)):
            i = 8*7*k
            for (x, y) in black_figs:
                if pos[x][y] == figs2[k]:
                    b = "{0:b}".format(8*x+y+1)
                    for j in range(0, len(b)):
                        if b[j] == '1':
                            res[i] = 0.99
                        i+=1
                        j+=1
        return numpy.array(res)
    def isDif(self, fr, to):
        fig_fr = self.board(fr)
        fig_to = self.board(to)
        return (isWhite(fig_fr) and isBlack(fig_to)) or (isWhite(fig_to) and isBlack(fig_fr))
    def updateProp(self):
        self.prop = [self.white_king, self.white_rook_right, self.white_rook_left, self.black_king, self.black_rook_right, self.black_rook_left, self.white_king_coords, self.black_king_coords, self.white, self.last]
    def updateFigs(self):
        self.white_figs.clear()
        self.black_figs.clear()
        for i in range(8):
            for j in range(8):
                if isWhite(self.board((i, j))):
                    self.white_figs.append((i, j))
                elif isBlack(self.board((i, j))):
                    self.black_figs.append((i, j))
    def canStep(self, fr, to):
        if fr[0] < 0 or fr[0] > 7 or to[0] < 0 or to[0] > 7 or fr[1] < 0 or fr[1] > 7 or to[1] < 0 or to[1] > 7:
            return False
        fig_fr = self.board(fr)
        fig_to = self.board(to)
        if fig_fr == '♙':
            if fr[1] == to[1]:
                if to[0] == fr[0]-1 and fig_to == ' ':
                    return True
                elif to[0] == fr[0]-2 and fig_to == ' ' and fr[0] == 6 and self.board((fr[0]-1, fr[1])) == ' ':
                    return True
            elif self.isDif(fr, to):
                return True
            elif fig_to == ' ' and self.board((to[0]+1, to[1])) == '♟' and self.last[1] == (to[0]+1, to[1]) and self.last[0] == (to[0]-1, to[1]):
                return True
            
        elif fig_fr == '♟':
            if fr[1] == to[1]:
                if to[0] == fr[0]+1 and self.board(to) == ' ':
                    return True
                elif to[0] == fr[0]+2 and self.board(to) == ' ' and fr[0] == 1 and self.board((fr[0]+1, fr[1])) == ' ':
                    return True
            elif self.isDif(fr, to):
                return True
            elif fig_to == ' ' and self.board((to[0]-1, to[1])) == '♙' and self.last[1] == (to[0]-1, to[1]) and self.last[0] == (to[0]+1, to[1]):
                return True
            
        elif fig_fr == '♔' and abs(fr[1] - to[1]) >=2:
            if not self.white_king and not self.white_rook_right and to[1]-fr[1] == 2 and to[1]-1 >= 0 and to[1]+1 < 8 and self.board((to[0], to[1]+1)) == '♖' and self.board((to[0], to[1]-1)) == ' ' and fig_to == ' ':
                fields = self.getFieldsOfBlack()
                if (to[0], to[1]-1) not in fields and to not in fields:
                    return True
            elif not self.white_king and not self.white_rook_left and fr[1]-to[1] == 2 and fr[1]-4 >= 0 and self.board((fr[0], fr[1]-4)) == '♖' and self.board((fr[0], fr[1]-3)) == ' ' and self.board((fr[0], fr[1]-2)) == ' ' and self.board((fr[0], fr[1]-1)) == ' ':
                fields = self.getFieldsOfBlack()
                if (fr[0], fr[1]-3) not in fields and (fr[0], fr[1]-2) not in fields and (fr[0], fr[1]-1) not in fields:
                    return True
                
        elif fig_fr == '♚' and abs(fr[1] - to[1]) >=2:
            if not self.black_king and not self.black_rook_right and to[1]-fr[1] == 2 and to[1]-1 >= 0 and to[1]+1 < 8 and self.board((to[0], to[1]+1)) == '♜' and self.board((to[0], to[1]-1)) == ' ' and fig_to == ' ':
                fields = self.getFieldsOfWhite()
                if (to[0], to[1]-1) not in fields and to not in fields:
                    return True
            elif not self.black_king and not self.black_rook_left and fr[1]-to[1] == 2 and fr[1]-4 >= 0 and self.board((fr[0], fr[1]-4)) == '♜' and self.board((fr[0], fr[1]-3)) == ' ' and self.board((fr[0], fr[1]-2)) == ' ' and self.board((fr[0], fr[1]-1)) == ' ':
                fields = self.getFieldsOfWhite()
                if (fr[0], fr[1]-3) not in fields and (fr[0], fr[1]-2) not in fields and (fr[0], fr[1]-1) not in fields:
                    return True
                
        elif fig_to == ' ' or self.isDif(fr, to):
            return True
        return False
    def getStepsByFig(self, cur, just_fields = False):
        fig = self.board(cur)
        steps = []
        if fig == '♙':
            if self.canStep(cur, (cur[0]-1, cur[1])):
                steps.append([cur, (cur[0]-1, cur[1])])
            if self.canStep(cur, (cur[0]-2, cur[1])):
                steps.append([cur, (cur[0]-2, cur[1])])
            if self.canStep(cur, (cur[0]-1, cur[1]+1)):
                steps.append([cur, (cur[0]-1, cur[1]+1)])
            if self.canStep(cur, (cur[0]-1, cur[1]-1)):
                steps.append([cur, (cur[0]-1, cur[1]-1)])
        elif fig == '♟':
            if self.canStep(cur, (cur[0]+1, cur[1])):
                steps.append([cur, (cur[0]+1, cur[1])])
            if self.canStep(cur, (cur[0]+2, cur[1])):
                steps.append([cur, (cur[0]+2, cur[1])])
            if self.canStep(cur, (cur[0]+1, cur[1]+1)):
                steps.append([cur, (cur[0]+1, cur[1]+1)])
            if self.canStep(cur, (cur[0]+1, cur[1]-1)):
                steps.append([cur, (cur[0]+1, cur[1]-1)])
        elif fig in '♖♜':
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1])):
                steps.append([cur, (cur[0]+i, cur[1])])
                if self.isDif(cur, (cur[0]+i, cur[1])):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1])):
                steps.append([cur, (cur[0]-i, cur[1])])
                if self.isDif(cur, (cur[0]-i, cur[1])):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0], cur[1]+i)):
                steps.append([cur, (cur[0], cur[1]+i)])
                if self.isDif(cur, (cur[0], cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0], cur[1]-i)):
                steps.append([cur, (cur[0], cur[1]-i)])
                if self.isDif(cur, (cur[0], cur[1]-i)):
                    break
                i+=1
        elif fig in "♗♝":
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1]+i)):
                steps.append([cur, (cur[0]+i, cur[1]+i)])
                if self.isDif(cur, (cur[0]+i, cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1]+i)):
                steps.append([cur, (cur[0]-i, cur[1]+i)])
                if self.isDif(cur, (cur[0]-i, cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1]-i)):
                steps.append([cur, (cur[0]+i, cur[1]-i)])
                if self.isDif(cur, (cur[0]+i, cur[1]-i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1]-i)):
                steps.append([cur, (cur[0]-i, cur[1]-i)])
                if self.isDif(cur, (cur[0]-i, cur[1]-i)):
                    break
                i+=1
            i = 1
        elif fig in "♕♛":
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1]+i)):
                steps.append([cur, (cur[0]+i, cur[1]+i)])
                if self.isDif(cur, (cur[0]+i, cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1]+i)):
                steps.append([cur, (cur[0]-i, cur[1]+i)])
                if self.isDif(cur, (cur[0]-i, cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1]-i)):
                steps.append([cur, (cur[0]+i, cur[1]-i)])
                if self.isDif(cur, (cur[0]+i, cur[1]-i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1]-i)):
                steps.append([cur, (cur[0]-i, cur[1]-i)])
                if self.isDif(cur, (cur[0]-i, cur[1]-i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]+i, cur[1])):
                steps.append([cur, (cur[0]+i, cur[1])])
                if self.isDif(cur, (cur[0]+i, cur[1])):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0]-i, cur[1])):
                steps.append([cur, (cur[0]-i, cur[1])])
                if self.isDif(cur, (cur[0]-i, cur[1])):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0], cur[1]+i)):
                steps.append([cur, (cur[0], cur[1]+i)])
                if self.isDif(cur, (cur[0], cur[1]+i)):
                    break
                i+=1
            i = 1
            while self.canStep(cur, (cur[0], cur[1]-i)):
                steps.append([cur, (cur[0], cur[1]-i)])
                if self.isDif(cur, (cur[0], cur[1]-i)):
                    break
                i+=1
        elif fig in "♘♞":
            if self.canStep(cur, (cur[0]-1, cur[1]-2)):
                steps.append([cur, (cur[0]-1, cur[1]-2)])
            if self.canStep(cur, (cur[0]-1, cur[1]+2)):
                steps.append([cur, (cur[0]-1, cur[1]+2)])
            if self.canStep(cur, (cur[0]+1, cur[1]-2)):
                steps.append([cur, (cur[0]+1, cur[1]-2)])
            if self.canStep(cur, (cur[0]+1, cur[1]+2)):
                steps.append([cur, (cur[0]+1, cur[1]+2)])
            if self.canStep(cur, (cur[0]-2, cur[1]-1)):
                steps.append([cur, (cur[0]-2, cur[1]-1)])
            if self.canStep(cur, (cur[0]-2, cur[1]+1)):
                steps.append([cur, (cur[0]-2, cur[1]+1)])
            if self.canStep(cur, (cur[0]+2, cur[1]-1)):
                steps.append([cur, (cur[0]+2, cur[1]-1)])
            if self.canStep(cur, (cur[0]+2, cur[1]+1)):
                steps.append([cur, (cur[0]+2, cur[1]+1)])
        elif fig in "♔♚":
            if self.canStep(cur, (cur[0], cur[1]+1)):
                steps.append([cur, (cur[0], cur[1]+1)])
            if self.canStep(cur, (cur[0], cur[1]-1)):
                steps.append([cur, (cur[0], cur[1]-1)])
            if self.canStep(cur, (cur[0]+1, cur[1])):
                steps.append([cur, (cur[0]+1, cur[1])])
            if self.canStep(cur, (cur[0]-1, cur[1])):
                steps.append([cur, (cur[0]-1, cur[1])])
            if self.canStep(cur, (cur[0]+1, cur[1]+1)):
                steps.append([cur, (cur[0]+1, cur[1]+1)])
            if self.canStep(cur, (cur[0]+1, cur[1]-1)):
                steps.append([cur, (cur[0]+1, cur[1]-1)])
            if self.canStep(cur, (cur[0]-1, cur[1]+1)):
                steps.append([cur, (cur[0]-1, cur[1]+1)])
            if self.canStep(cur, (cur[0]-1, cur[1]-1)):
                steps.append([cur, (cur[0]-1, cur[1]-1)])
            if not just_fields:
                if self.canStep(cur, (cur[0], cur[1]+2)):
                    steps.append([cur, (cur[0], cur[1]+2)])
                if self.canStep(cur, (cur[0], cur[1]-2)):
                    steps.append([cur, (cur[0], cur[1]-2)])
        return steps
    def getFieldsOfWhite(self):
        steps = []
        for (x, y) in self.white_figs:
            steps+=self.getStepsByFig((x, y), True)
        fields = set()
        for [fr, to] in steps:
            fields.add(to)
        return fields
    def getFieldsOfBlack(self):
        steps = []
        for (x, y) in self.black_figs:
            steps+=self.getStepsByFig((x, y), True)
        fields = set()
        for [fr, to] in steps:
            fields.add(to)
        return fields
    def getStepsOfWhite(self):
        steps = []
        for (x, y) in self.white_figs:
            steps+=self.getStepsByFig((x, y))
        return steps
    def getStepsOfBlack(self):
        steps = []
        for (x, y) in self.black_figs:
            steps+=self.getStepsByFig((x, y))
        return steps
    def isCheck(self, isWhite):
        if isWhite:
            #print(self.pos)
            fields = self.getFieldsOfWhite()
            #print(fields)
            if self.black_king_coords in fields:
                return True
        else:
            fields = self.getFieldsOfBlack()
            if self.white_king_coords in fields:
                return True
        return False
    def isMate(self, isWhite):
        if isWhite:
            if self.isCheck(isWhite):
                for [fr, to] in self.getStepsOfBlack():
                    arr = self.virtualStep(fr, to)
                    virtual_game = chess(False, arr[0], arr[1])
                    if not virtual_game.isCheck(isWhite):
                        return False
                return True
        else:
            if self.isCheck(isWhite):
                for [fr, to] in self.getStepsOfWhite():
                    arr = self.virtualStep(fr, to)
                    virtual_game = chess(False, arr[0], arr[1])
                    if not virtual_game.isCheck(isWhite):
                        return False
                return True
        return False
    def isDraw(self, isWhite):
        steps = []
        if isWhite:
            got_steps = self.getStepsOfBlack()
            for [fr, to] in got_steps:
                if self.isHelpfulStep(fr, to, not self.white):
                    steps.append([fr, to])
            if not steps:
                return True
        else:
            got_steps = self.getStepsOfWhite()
            for [fr, to] in got_steps:
                if self.isHelpfulStep(fr, to, not self.white):
                    steps.append([fr, to])
            if not steps:
                return True
        return False
    def isHelpfulStep(self, fr, to, isWhite):
        arr = self.virtualStep(fr, to)
        virtual_game = chess(False, arr[0], arr[1])
        if not virtual_game.isCheck(isWhite):
            return True
        return False
    def getSteps(self):
        steps = []
        status = self.getStatus()
        if status == "ingame":
            if self.white:
                for (x, y) in self.white_figs:
                    got_steps = self.getStepsByFig((x, y))
                    for [fr, to] in got_steps:
                        if self.isHelpfulStep(fr, to, not self.white):
                            steps.append([fr, to])
            else:
                for (x, y) in self.black_figs:
                    got_steps = self.getStepsByFig((x, y))
                    for [fr, to] in got_steps:
                        if self.isHelpfulStep(fr, to, not self.white):
                            steps.append([fr, to])
        
        return steps
    def step(self, fr, to):
        if self.isValid(fr, to):
            temp = self.board(fr)
            temp_to = self.board(to)
            self.pos[fr[0]][fr[1]] = ' '
            self.pos[to[0]][to[1]] = temp
            if temp in "♙♟":
                if to[0] == 0 or to[0] == 7:
                    if temp == '♙':
                        self.pos[to[0]][to[1]] = '♕'
                    else:
                        self.pos[to[0]][to[1]] = '♛'
                elif abs(to[0]-fr[0]) == 1 and abs(to[1]-fr[1]) == 1 and temp_to == ' ':
                    if temp == '♙':
                        self.pos[to[0]+1][to[1]] = ' '
                    else:
                        self.pos[to[0]-1][to[1]] = ' '
                        
            elif temp == '♔':
                self.white_king = True
                self.white_king_coords = (to[0], to[1])
                if to[1]-fr[1] == 2:
                    temp2 = self.board((to[0], to[1]+1))
                    self.pos[to[0]][to[1]+1] = ' '
                    self.pos[to[0]][to[1]-1] = temp2
                elif fr[1]-to[1] == 2:
                    temp2 = self.board((fr[0], fr[1]-4))
                    self.pos[fr[0]][fr[1]-4] = ' '
                    self.pos[to[0]][to[1]+1] = temp2
            elif temp == '♚':
                self.black_king = True
                self.black_king_coords = (to[0], to[1])
                if to[1]-fr[1] == 2:
                    temp2 = self.board((to[0], to[1]+1))
                    self.pos[to[0]][to[1]+1] = ' '
                    self.pos[to[0]][to[1]-1] = temp2
                elif fr[1]-to[1] == 2:
                    temp2 = self.board((fr[0], fr[1]-4))
                    self.pos[fr[0]][fr[1]-4] = ' '
                    self.pos[to[0]][to[1]+1] = temp2
            elif temp == '♖':
                if fr == (7, 0):
                    self.white_rook_left = True
                elif fr == (7, 7):
                    self.white_rook_right = True
            elif temp == '♜':
                if fr == (0, 0):
                    self.black_rook_left = True
                elif fr == (0, 7):
                    self.black_rook_right = True
            
            self.last = [fr, to]
            self.white = not self.white
            self.updateProp()
            self.updateFigs()
            self.history.append(self.toArray(self.pos, self.white_figs, self.black_figs))
                
            if self.vis:
                clear_output()
                print(self.pos)
                status = self.getStatus()
                if status != "ingame":
                    print(status)
            return True
        return False
    def getStatus(self):
        if (not self.isCheck(not self.white)) and self.isDraw(not self.white):
            return "draw"
        if self.isMate(not self.white):
            if not self.white:
                return "white"
            else:
                return "black"
        return "ingame"
    def virtualStep(self, fr, to):
        pos = self.pos.copy()
        prop = self.prop.copy()
        
        temp = pos[fr[0]][fr[1]]
        temp_to = pos[to[0]][to[1]]
        pos[fr[0]][fr[1]] = ' '
        pos[to[0]][to[1]] = temp
        
        if temp in "♙♟":
            if to[0] == 0 or to[0] == 7:
                if temp == '♙':
                    pos[to[0]][to[1]] = '♕'
                else:
                    pos[to[0]][to[1]] = '♛'
            elif abs(to[0]-fr[0]) == 1 and abs(to[1]-fr[1]) == 1 and temp_to == ' ':
                if temp == '♙':
                    pos[to[0]+1][to[1]] = ' '
                else:
                    pos[to[0]-1][to[1]] = ' '
        elif temp in "♔♚":
            if temp == '♔':
                prop[0] = True
                prop[6] = (to[0], to[1])
            else:
                prop[3] = True
                prop[7] = (to[0], to[1])
                
            if to[1]-fr[1] == 2:
                temp2 = pos[to[0]][to[1]+1]
                pos[to[0]][to[1]+1] = ' '
                pos[to[0]][to[1]-1] = temp2
            elif fr[1]-to[1] == 2:
                temp2 = pos[to[0]][to[1]-4]
                pos[fr[0]][fr[1]-4] = ' '
                pos[to[0]][to[1]+1] = temp2
        elif temp == '♖':
            if fr == (7, 0):
                prop[1] = True
            elif fr == (7, 7):
                prop[2] = True
        elif temp == '♜':
            if fr == (0, 0):
                prop[5] = True
            elif fr == (0, 7):
                prop[4] = True
        
        prop[8] = not self.white
        prop[9] = [fr, to]
        return [pos, prop]
    def isValid(self, fr, to):
        steps = self.getSteps()
        if [fr, to] in steps:
            return True
        return False
    def stepAI(self):
        steps = self.getSteps()
        sel = [(0, 0), (0, 0)]
        M = 1000.0
        if self.white:
            M = 0.0
        for [fr, to] in steps:
            arr = self.virtualStep(fr, to)
            virtual_game = chess(False, arr[0], arr[1])
            inputs = self.toArray(arr[0], virtual_game.white_figs, virtual_game.black_figs)
            out = AI.query(inputs)
            outputs = out[0][0]
            #print(outputs)
            if not self.white:
                if outputs < M:
                    M = outputs
                    sel = [fr, to]
            else:
                if outputs > M:
                    M = outputs
                    sel = [fr, to]
        self.step(sel[0], sel[1])
        #print(M)
    def start(self, you_white = True, AI = False):
        if AI:
            while True:
                if self.white == you_white:
                    print("Player")
                    a, b, c, d = list(map(int,input().split())) 
                    self.step((a, b), (c, d))
                else:
                    print("AI")
                    steps = self.getSteps()
                    #print(steps)
                    sel = [(0, 0), (0, 0)]
                    M = 0
                    if you_white:
                        M = 1000
                    for [fr, to] in steps:
                        pos, wf, bf = self.virtualStep(fr, to)
                        inputs = self.toArray(pos, wf, bf)
                        outputs = AI.query(inputs)
                        if you_white:
                            if outputs[0] < M:
                                M = outputs[0]
                                sel = [fr, to]
                        else:
                            if outputs[0] > M:
                                M = outputs[0]
                                sel = [fr, to]
                    self.step(sel[0], sel[1])
                    
        else:
            while True:
                if self.white:
                    print("Player#1")
                else:
                    print("Player#2")
                a, b, c, d = list(map(int,input().split())) 
                game.step((a, b), (c, d))

In [3]:
class neuralNetwork:
    def __init__(self, nodes, lr):
        self.nodes = nodes
        self.lr = lr
        self.layers = len(nodes)
        
        self.W = [numpy.random.normal(0.0, pow(self.nodes[i+1], -0.5), (self.nodes[i+1], self.nodes[i])) for i in range(0, self.layers-1)]
        
        self.activation_function = lambda x: scipy.special.expit(x)
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        outputs = inputs
        outputs_array = [outputs]
        
        for i in range(0, self.layers-1):
            outputs = self.activation_function(numpy.dot(self.W[i], outputs))
            outputs_array.append(outputs)
        
        errors = targets-outputs
        
        for i in range(self.layers-2, -1, -1):
            self.W[i] += self.lr * ((errors * outputs_array[i+1] * (1.0 - outputs_array[i+1]))) * (numpy.transpose(outputs_array[i]))
            errors = numpy.dot(self.W[i].T, errors)
        
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        outputs = inputs
        
        for i in range(0, self.layers-1):
            outputs = self.activation_function(numpy.dot(self.W[i], outputs))
        
        return outputs
    def saveW(self):
        for i in range(self.layers-1):
            numpy.savetxt("weights/W"+str(i+1), self.W[i])
    def loadW(self):
        for i in range(self.layers-1):
            self.W[i] = numpy.loadtxt("weights/W"+str(i+1))
            if len(self.W[i].shape) == 1:
                self.W[i] = numpy.array([self.W[i]])
    def clearW(self):
        self.W = [numpy.random.normal(0.0, pow(self.nodes[i+1], -0.5), (self.nodes[i+1], self.nodes[i])) for i in range(0, self.layers-1)]

In [4]:
#Данные нейросети
nodes = [82*7, 300, 1]
learning_rate = 0.3

AI = neuralNetwork(nodes, learning_rate)
AI.loadW()

def learn(game, win):
    K = 0.5
    if win == "white":
        K = 0.8
    else:
        K = 0.1
    for i in range(0, len(game.history)):
        inputs = game.history[i]
        outputs = [[K]]
        AI.train(inputs, outputs)

In [ ]:
#Реализация сервера
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "127.0.0.1"
port = 1234
soc.bind((host, port))
soc.listen(10)

USERS_me = dict()
USERS_player = dict()
USERS_ai = dict()

class User(threading.Thread):
    def __init__(self, num, con):
        super().__init__(target=None, name=("TH"+num), args=(con,))
        self.num = num
        self.con = con
    def run(self):
        while(True):
            #try:
                data = self.con.recv(1024).decode()
                arr = data.split("|")
                if len(arr) == 4 and arr[3] == "remove":
                    print(data)
                    username = arr[0]
                    who = arr[1]
                    typ = arr[2]
                    if who == "me":
                        USERS_me[username] = [chess(False), typ, "ingame"]
                        if USERS_me[username][1] == "white":
                            self.con.send(toString(USERS_me[username][0].getPos()).encode('utf-8'))
                        else:
                            self.con.send(toString(USERS_me[username][0].getReversePos()).encode('utf-8'))
                    elif who == "ai":
                        USERS_ai[username] = [chess(False), typ, "ingame", [(-1, -1), (-1, -1)]]
                        if USERS_ai[username][1] == "white":
                            self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                        else:
                            self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                if ',' not in data:
                    username = arr[0]
                    who = arr[1]
                    if who == "me":
                        typ = arr[2]
                        if username not in USERS_me:
                            USERS_me[username] = [chess(False), typ, "ingame"]
                        USERS_me[username][1] = typ
                        status = USERS_me[username][2]
                        if USERS_me[username][1] == "white":
                            self.con.send((toString(USERS_me[username][0].getPos())+"|"+status).encode('utf-8'))
                        else:
                            self.con.send((toString(USERS_me[username][0].getReversePos())+"|"+status).encode('utf-8'))
                            
                        if USERS_me[username][2] != "ingame":
                            USERS_me[username] = [chess(False), typ, "ingame"]
                    elif who == "ai":
                        ul = arr[2]
                        if username not in USERS_ai:
                            rand = random.randrange(0, 2)
                            typ = "white"
                            if rand == 1:
                                typ = "black"
                            USERS_ai[username] = [chess(False), typ, "ingame", [(-1, -1), (-1, -1)]]
                        #USERS_ai[username][1] = typ
                        
                        if USERS_ai[username][3] != USERS_ai[username][0].last or ul == "load":
                            print(data)
                            USERS_ai[username][3] = USERS_ai[username][0].last
                            status = USERS_ai[username][2]
                            if USERS_ai[username][1] == "white":
                                self.con.send((toString(USERS_ai[username][0].getPos())+"|"+status+"|"+USERS_ai[username][1]).encode('utf-8'))
                            else:
                                if USERS_ai[username][3] == [(-1, -1), (-1, -1)]:
                                    USERS_ai[username][0].stepAI()
                                self.con.send((toString(USERS_ai[username][0].getReversePos())+"|"+status+"|"+USERS_ai[username][1]).encode('utf-8'))
                        else:
                            self.con.send("No change".encode('utf-8'))
                        if USERS_ai[username][2] != "ingame":
                            learn(USERS_ai[username][0], status)
                            AI.saveW()
                            rand = random.randrange(0, 2)
                            typ = "white"
                            if rand == 1:
                                typ = "black"
                            USERS_ai[username] = [chess(False), typ, "ingame", [(-1, -1), (-1, -1)]]
                    elif who == "player":
                        what = arr[2]
                else:
                    print(data)
                    username = arr[0]
                    who = arr[1]
                    arr_step = arr[2].split(",")
                    
                    a = int(arr_step[0])
                    b = int(arr_step[1])
                    c = int(arr_step[2])
                    d = int(arr_step[3])
                    
                    if who == "me":
                        if USERS_me[username][0].step((a,b), (c,d)):
                            status = USERS_me[username][0].getStatus()
                            if status == "ingame":
                                if USERS_me[username][1] == "white":
                                    self.con.send(toString(USERS_me[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_me[username][0].getReversePos()).encode('utf-8'))
                            else:
                                print(status)
                                USERS_me[username][2] = status
                                self.con.send(status.encode('utf-8'))
                        else:
                            print("impossible")
                            self.con.send("impossible".encode('utf-8'))
                    elif who == "ai":
                        if USERS_ai[username][0].step((a,b), (c,d)):
                            status = USERS_ai[username][0].getStatus()
                            if status == "ingame":
                                if USERS_ai[username][1] == "white":
                                    self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                                
                                USERS_ai[username][0].stepAI()
                                if USERS_ai[username][1] == "white":
                                    self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                            else:
                                print(status)
                                USERS_ai[username][2] = status
                                self.con.send(status.encode('utf-8'))
                        else:
                            print("impossible")
                            self.con.send("impossible".encode('utf-8'))
                        
            #except:
                #print("Пользователь отключился")
                return

while(True):
    conn, addr = soc.accept()
    #print("Пользователь подключился: ",addr)
    TH = User(str(addr), conn)
    TH.start()

test|me|6,4,5,4
test|me|1,4,2,4
test|ai|load
0.9999998652668276
0.9999998700960362
0.9999998494247344
0.9999998543095053
0.9999998652668276
0.9999998700960362
0.999999874490676
0.9999998793931849
0.9999999019942877
0.9999999066674155
0.9999999098904876
0.9999999145641731
0.9999999263057141
0.9999999305405359
0.9999999331619884
0.9999999451762255
0.9999998289039274
0.9999998289039274
0.999999717796238
0.9999997521504286
test|ai|update
test|ai|1,4,2,4
test|ai|update0.9999999457312677

test|ai|load
0.9999999421362604
0.9999999508068134
0.9999999421362604
0.9999999456208518
0.9999999421362604
0.9999999508068134
0.999999940685858
0.9999999525319736
0.9999999402862327
0.9999999566163783
0.9999999436450787
0.9999999549538987
0.9999999422963699
0.9999999580079755
0.9999999398584588
0.9999999398584588
0.9999999083868172
0.9999999190890344
0.9999999521851558
0.9999999534907645
test|ai|update
test|ai|0,5,3,2
test|ai|load0.9999999040289124

0.9999999460515734
0.999999963883899
0.9999999579691785
0

test|ai|update
test|ai|0,5,3,2
0.059758702338737535
test|ai|load
0.10663470405748773
0.12015325483495144
0.11699340496120002
0.12015325483495144
0.1049657911588578
0.12015325483495144
0.11699340496120002
0.10203851816262384
0.09151459501576793
0.10817218957439861
0.10817218957439861
0.08343773712063687
0.0955934612625852
0.11083659767000682
0.11083659767000682
0.09803471289388424
0.10663470405748773
0.0989172570600528
0.0989172570600528
0.10378034327037028
0.09500156438346045
0.10002265215503756
0.0964477563390051
0.09061410797042475
test|ai|update
test|ai|0,3,2,5
0.0679563274771496test|ai|load

0.11699340496120002
0.11635334663946344
0.11665187545068842
0.10102577383678514
0.11665187545068842
0.11343833765242989
0.10073145644187807
0.08697652100924758
0.10753343365099108
0.10406086398877032
0.0955934612625852
0.0915641010632521
0.12922161119816808
0.1253223668183265
0.1153326043977423
0.12015325483495144
0.11239076771085382
0.11239076771085382
0.11734148308538492
0.10810903731724403
0

test|ai|update
test|ai|1,4,3,4
0.8424125414143345
test|ai|load
0.8353364838893823
0.8424125414143345
0.8182793911578016
0.8424125414143345
0.8353364838893823
0.8617718252854858
0.8475470215142901
0.868552769134758
0.868552769134758
0.864903595665082
0.864903595665082
0.8394276841893341
0.8499983902580479
0.8403257436991153
0.8403257436991153
0.8256666745595614
0.831016463909464
0.8230875076730243
0.825296634826921
0.8330031775137603
0.8247258965394672
0.8205697129862217
0.8284393546493681
0.821279370709795
0.8253749534608574
0.8251055815043888
0.8157531615981121
0.8170803714436992
test|ai|update
test|ai|0,3,2,5
0.868552769134758
test|ai|load
0.8818064901652061
0.8759210971877992
0.8818064901652061
0.8631983745881009
0.8818064901652061
0.8759210971877992
0.857191047960094
0.8432882385442597
0.8717489857510017
0.8717489857510017
0.8395382459036076
0.8569508915207887
0.8751896706730591
0.8751896706730591
0.8639495781380845
0.868552769134758
0.8746728585171507
0.8622445270684675
0.85966837

test|ai|update
test|ai|5,4,4,4
test|ai|load0.8054170587046657

0.8061971541921807
0.8006135096699902
0.8006135096699902
0.8006135096699902
0.8006135096699902
0.819769357358724
0.8006135096699902
0.8327709714310185
0.8263549305446772
0.8327709714310185
0.8000947793885298
0.7924515183535809
0.799331648535701
0.8165352836752833
0.8001690346351447
0.7876435086842623
0.7876435086842623
test|ai|update
test|ai|5,1,7,0
impossible
test|ai|7,0,4,0
0.8542076685655893
test|ai|load
0.85489798840678
0.85489798840678
0.8725053771253236
0.8417103901694144
0.8417103901694144
0.8417103901694144
0.8417103901694144
0.8595881896112554
0.8417103901694144
0.8376224382742948
0.8231454265276664
0.8376224382742948
0.8231454265276664
0.7973084731106076
0.8025307696019194
0.8375289867309738
0.8471103503971469
0.797528153409702
test|ai|update
test|ai|4,0,3,5
impossible
test|ai|3,5,2,4
test|ai|load
0.8413474198055503
0.8412328401013334
0.8412328401013334
0.8602010511509703
0.8285911869420297
0.8285911869420297
0.82

test|ai|update
test|ai|3,3,1,5
white
test|ai|load
test|ai|load
0.8180924160163777
0.8247020753140045
0.8059036156116621
0.8121693718062368
0.8180924160163777
0.8247020753140045
0.811574748458177
0.8182985792518819
0.8362750656393052
0.8434763216552637
0.8277081009304827
0.8348288809890294
0.8403935465866877
0.8476051418733374
0.8002014097031007
0.8121870625024145
0.8149056575896569
0.8149056575896569
0.7982864451949407
0.7971509442083504
test|ai|update
test|ai|1,4,2,4
0.8533232220187429
test|ai|load0.855643311879382

0.8490532134964452
0.855643311879382
0.8372228397736274
0.855643311879382
0.8490532134964452
0.8750414027367328
0.8654523014225644
0.8844954456855247
0.8844954456855247
0.898579746142001
0.8909555168109611
0.8902814936589463
0.9016784949360775
0.8486945665140838
0.8486945665140838
0.8262861705841746
0.8231397975308391
0.836300101599456
0.8345843407917211
test|ai|update
test|ai|0,3,2,5
0.851997389744796
test|ai|load
0.8880943786937234
0.9075739425779392
0.9020866721262285
0

...

test|me|white|remove
test|ai|load


Exception in thread TH('127.0.0.1', 25077):
Traceback (most recent call last):
  File "c:\users\пк\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-5-c98db80031f4>", line 49, in run
ConnectionAbortedError: [WinError 10053] Программа на вашем хост-компьютере разорвала установленное подключение


test|me|white|remove
test|me|6,3,5,3
test|me|1,3,3,3
test|me|7,5,5,5
impossible


Exception in thread TH('127.0.0.1', 25108):
Traceback (most recent call last):
  File "c:\users\пк\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-5-c98db80031f4>", line 49, in run
ConnectionAbortedError: [WinError 10053] Программа на вашем хост-компьютере разорвала установленное подключение


test|me|white|remove
test|ai|load


In [ ]:
#Игра без ИИ
game = chess(True)
game.start(False)
